In [1]:
import cdsapi
import requests
import os

In [ ]:
#What a permaculture project need to know when they start

In [74]:
import time
from geopy.geocoders import Nominatim

# select the years you want to download:
start_year = 1992
end_year = 2022
year_range = [i for i in range(start_year, end_year + 1)]
print(year_range)

# Use geopy to get the latitude and longitude of the city
geolocator = Nominatim(user_agent="permaculture-climate")
location = geolocator.geocode("Puebla de don Fadrique, Spain")
# Add a delay between requests
time.sleep(1)

c = cdsapi.Client()
try:
    data = c.retrieve("reanalysis-era5-single-levels-monthly-means",
    {"format": "grib",
     "product_type": "monthly_averaged_reanalysis_by_hour_of_day",
     "variable": ['10m_u_component_of_wind', '10m_v_component_of_wind', 
                '2m_temperature',
                'snowfall', 
                'soil_temperature_level_1', 
                'soil_type',
                'total_cloud_cover', 
                'total_precipitation',
                ],
    "area": [location.latitude + 1, 
             location.longitude - 1, 
             location.latitude - 1, 
             location.longitude + 1],  # North, West, South, East. 
    "year": year_range,
    "month": ['01', '02', '03',
           '04', '05', '06',
           '07', '08', '09',
           '10', '11', '12'],
    "time": ["00:00","01:00","02:00","03:00","04:00","05:00",
             "06:00","07:00","08:00","09:00","10:00","11:00",
             "12:00", "13:00","14:00","15:00","16:00","17:00",
             "18:00","19:00","20:00","21:00","22:00","23:00"]
    })

    # Get the location of the file to download
    url = data.location

    # Download the file
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()

except requests.exceptions.HTTPError as errh:
    print ("HTTP Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)
except requests.exceptions.RequestException as err:
    print ("Something went wrong with the request:",err)

else:
    # If the request was successful, write the file
    filename = 'download.grib'
    with open(filename, 'wb') as f:
        f.write(response.content)

    # Print the location where the file is saved
    print(f"File saved at: {os.path.abspath(filename)}")

[1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


2023-11-24 10:43:49,887 INFO Welcome to the CDS
2023-11-24 10:43:49,888 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2023-11-24 10:43:50,054 INFO Request is queued
2023-11-24 10:43:52,646 INFO Request is running
2023-11-24 11:50:13,895 INFO Request is completed


File saved at: /Users/giacomo/Documents/projects/permaculture-climate/download.grib


In [ ]:
#temperatura durante l'anno 
#precipitaciones durante l'anno
#vento durante l'anno (rosa dei venti)

#x axisx: month name, tilted, on the 15th of each month, plotly express theme "simple_white"

In [4]:
#use pygrib to read the variables names
import pygrib

# Open the GRIB file
grbs = pygrib.open('download.grib')

# Read the names of the variables
variable_names = [grb.name for grb in grbs]

# Convert the set to a list
unique_variable_names = list(set(variable_names))

# Print the variable names
print(unique_variable_names)


KeyboardInterrupt: 

In [9]:
# Open the GRIB file
grbs = pygrib.open('download.grib')

# Select the 'Total cloud cover' variable
total_cloud_cover = [grb for grb in grbs if grb.name == 'Total cloud cover']

# Print the first instance of 'Total cloud cover'
print(total_cloud_cover[0])

# Get the data values for the first instance of 'Total cloud cover'
data_values = total_cloud_cover

# Print the data values
print(data_values)

7:Total cloud cover:(0 - 1) (avgua):regular_ll:surface:level 0:fcst time 0 hrs (avgua):from 199201010000


AttributeError: 'list' object has no attribute 'values'

In [8]:
# Get the dimensions of the data
dimensions = data_values.shape
dimensions

(9, 9)

In [ ]:
import xarray as xr

# List of variables to load
variables = ['Total cloud cover', 'Soil temperature level 1', '10 metre V wind component', 
             '2 metre temperature', 'Soil type', 'Total precipitation', '10 metre U wind component', 'Snowfall']

# Dictionary to hold the datasets
datasets = {}

# Open the GRIB file for each variable
for var in variables:
    ds = xr.open_dataset('download.grib', engine='cfgrib', backend_kwargs={'filter_by_keys': {'parameterName': var}})
    datasets[var] = ds

# Print the datasets
for var, ds in datasets.items():
    print(f"{var}:")
    print(ds)
    print("\n")


In [11]:
# Print the units for 'Total precipitation'
print(f"Units for Total precipitation: {datasets['10 metre U wind component']['u10'].units}")

Units for Total precipitation: m s**-1


In [12]:
import pandas as pd
# Check if the time values for Total precipitation are always 6 and 18
time_values = pd.to_datetime(datasets['Total precipitation']['time'].values)
valid_times = [time for time in time_values if time.hour not in [6, 18]]

# Print the result
if valid_times:
    print(f"The following time values for Total precipitation are not 6 or 18: {valid_times}")
else:
    print("All time values for Total precipitation are 6 or 18.")

len(time_values)


All time values for Total precipitation are 6 or 18.


1116

In [13]:
# Calculate the climatology and average over latitude and longitude
precip_climatology = datasets['Total precipitation']['tp'].groupby('time.month').mean(['time', 'latitude', 'longitude', 'step'])*1000
# Calculate the average, maximum, and minimum temperature
avg_temp = datasets['2 metre temperature']['t2m'].groupby('time.month').mean(['time', 'latitude', 'longitude'])-273.15
max_temp = datasets['2 metre temperature']['t2m'].groupby('time.month').max(['time', 'latitude', 'longitude'])-273.15
min_temp = datasets['2 metre temperature']['t2m'].groupby('time.month').min(['time', 'latitude', 'longitude'])-273.15

#check the diemsnsions of the data
print(avg_temp.dims)
print(precip_climatology.dims)


('month',)
('month',)


In [45]:

from plotly.subplots import make_subplots

# Create a subplot with shared x-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add a bar chart for precipitation to the secondary y-axis
fig.add_trace(
    go.Bar(x=precip_climatology.month, y=precip_climatology, name='Precipitation', opacity=0.5),
    secondary_y=False,
)

# Add a line chart for temperature to the primary y-axis
fig.add_trace(
    go.Scatter(x=avg_temp.month, y=avg_temp, mode='lines', name='Temperature'),
    secondary_y=True,
)

# Set the layout to have two y-axes
fig.update_layout(
    yaxis=dict(title='Temperature'),
    yaxis2=dict(title='Precipitation', overlaying='y', side='right'),
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=avg_temp.month,
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)

# Show the figure
fig.show()

In [61]:
# Create a Plotly figure
fig = go.Figure()

# Add a line chart for average temperature
fig.add_trace(go.Scatter(
    x=avg_temp.month,
    y=avg_temp,
    mode='lines',
    name='Average Temperature'
))

# Add a shaded area for min-max temperature
fig.add_trace(go.Scatter(
    x=max_temp.month,
    y=max_temp,
    mode='lines',
    name='Max Temperature',
    line_color='red',
    fill=None
))

fig.add_trace(go.Scatter(
    x=min_temp.month,
    y=min_temp,
    mode='lines',
    name='Min Temperature',
    line_color='red',
    fill='tonexty',
))

fig.add_trace(go.Scatter(
    x=min_temp.month,
    y=min_temp,
    mode='lines',
    name='Min Temperature',
    line_color='red',
    fill='tozeroy',
))



# Set the layout
fig.update_layout(
    yaxis=dict(title='Temperature (°C)'),
    title='Average Temperature Data',
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=avg_temp.month,
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)

# Show the figure
fig.show()


In [68]:
import plotly.express as px

# Create a DataFrame from the DataArrays
df = pd.DataFrame({
    'month': avg_temp.month.values,
    'avg_temp': avg_temp.values,
    'max_temp': max_temp.values,
    'min_temp': min_temp.values
})

# Create a line chart for average temperature
fig = px.line(df, x='month', y='avg_temp', title='Monthly temperature ranges')

# Add a line chart for max temperature
fig.add_trace(go.Scatter(x=df['month'], y=df['max_temp'], mode='lines', name='Max Temperature', line_color='red'))

# Add a line chart for min temperature
fig.add_trace(go.Scatter(x=df['month'], y=df['min_temp'], mode='lines', name='Min Temperature', line_color='red', fill='tonexty'))

fig.add_hline(y=0, line_color='black', line_width=2)

# Add a line chart for min temperature
fig.add_trace(go.Scatter(x=df['month'], y=df['min_temp'], mode='lines', name='Min Temperature', line_color='red', fill='tonexty'))

# Set the layout
fig.update_layout(
    yaxis=dict(title='Temperature (°C)'),
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=df['month'],
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)

# Show the figure
fig.show()

In [53]:
import numpy as np

# Create masks for the portions of min and max temperature that are below zero
min_temp_below_zero = np.where(min_temp < 0, min_temp, 0)
max_temp_below_zero = np.where(max_temp < 0, max_temp, 0)

# Add a line chart for average temperature
fig.add_trace(go.Scatter(
    x=avg_temp.month,
    y=avg_temp,
    mode='lines',
    name='Average Temperature'
))

# Add a shaded area for min-max temperature
fig.add_trace(go.Scatter(
    x=max_temp.month,
    y=max_temp,
    mode='lines',
    name='Max Temperature',
    line_color='red',
    fill=None
))

fig.add_trace(go.Scatter(
    x=min_temp.month,
    y=min_temp,
    mode='lines',
    name='Min Temperature',
    line_color='red',
    fill='tonexty',
))

# Add a shaded area for the portion of min-max temperature that is below zero
fig.add_trace(go.Scatter(
    x=max_temp.month,
    y=max_temp_below_zero,
    mode='lines',
    line_color='blue',
    fill=None
))

fig.add_trace(go.Scatter(
    x=min_temp.month,
    y=min_temp_below_zero,
    mode='lines',
    line_color='blue',
    fill='tonexty',
))

# Set the layout
fig.update_layout(
    yaxis=dict(title='Temperature (°C)'),
    title='Average Temperature Data',
    xaxis=dict(
        title='Month',
        tickmode='array',
        tickvals=avg_temp.month,
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        tickangle=-45
    ),
    template='simple_white'
)

# Show the figure
fig.show()

In [83]:
# Calculate the temperature difference
temp_diff = max_temp - min_temp

# Create a Plotly figure
fig = go.Figure()

# Add a line chart for temperature difference
fig.add_trace(go.Scatter(
    x=temp_diff.month,
    y=temp_diff,
    mode='lines',
    name='Temperature Difference'
))

# Set the layout
fig.update_layout(
    xaxis=dict(title='Month'),
    yaxis=dict(title='Temperature Difference (°C)'),
    title='Temperature Difference Data'
)

# Show the figure
fig.show()

In [84]:
# Create a Plotly figure
fig = go.Figure()

# Add a bar chart for min-max temperature
fig.add_trace(go.Bar(
    x=min_temp.month,
    y=max_temp - min_temp,
    base=min_temp,
    name='Min-Max Temperature'
))

# Set the layout
fig.update_layout(
    xaxis=dict(title='Month'),
    yaxis=dict(title='Temperature (°C)'),
    title='Min-Max Temperature Data'
)

# Show the figure
fig.show()

In [85]:
# Subtract the average temperature from min and max temperatures
min_temp_adj = min_temp - avg_temp
max_temp_adj = max_temp - avg_temp

# Create a Plotly figure
fig = go.Figure()

# Add a bar chart for adjusted min-max temperature
fig.add_trace(go.Bar(
    x=min_temp_adj.month,
    y=max_temp_adj - min_temp_adj,
    base=min_temp_adj,
    name='Adjusted Min-Max Temperature'
))

# Set the layout
fig.update_layout(
    xaxis=dict(title='Month'),
    yaxis=dict(title='Temperature Difference from Average (°C)'),
    title='Adjusted Min-Max Temperature Data'
)

# Show the figure
fig.show()

In [69]:
import pandas as pd
import numpy as np
import plotly.express as px
import xarray as xr


# Convert xarray DataArrays to pandas DataFrame
df = pd.DataFrame({'speed': wind_speed.values, 'direction': wind_direction.values})

# Bin the data in terms of direction and wind speed to obtain frequencies
direction_bins = pd.cut(df['direction'], bins=np.arange(-180, 180.1, 22.5), right=True).astype(str)
speed_bins = pd.cut(df['speed'], bins=np.arange(0, df['speed'].max() + 1.1, 1), right=True).astype(str)

# Calculate frequencies
frequency_df = df.groupby([direction_bins, speed_bins]).size().reset_index(name='frequency')

# Calculate percentages
frequency_df['percentage'] = frequency_df['frequency'] / frequency_df['frequency'].sum() * 100

# Create the wind rose chart
fig = px.bar_polar(frequency_df, 
                   r='percentage',  # Change 'frequency' to 'percentage'
                   theta='direction', 
                   color='speed', 
                   template='',
                   color_continuous_scale='RdBu')  # Change color scale to 'RdBu'

# Update the layout to make it rectangular
fig.update_layout(
    width=800,  # Set the width to 700 pixels
    height=600,  # Set the height to 1000 pixels
    polar_angularlabel_text = frequency_df['direction']  # Use 'direction' as labels instead of degrees
)

fig.show()


NameError: name 'wind_speed' is not defined

In [125]:
sum(np.isnan(df['speed']))

0

In [145]:
df['speed'].describe()


count    8928.000000
mean        0.890333
std         0.615138
min         0.005879
25%         0.436278
50%         0.753375
75%         1.197439
max         4.143425
Name: speed, dtype: float64

In [ ]:
#soil temperature

In [ ]:
#cloud cover/sunny days

In [ ]:
#hours of sun from latitude

In [ ]:
#ground freezing another plot or highlight in temperature plot